# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [6]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [55]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [280]:
group = data.groupby(by=(["CustomerID","ProductName"])).sum()

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [281]:
pivot = pd.pivot_table(group, index="ProductName",columns="CustomerID",values="Quantity").fillna(0)
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [282]:
from scipy.spatial.distance import pdist, squareform


matrix = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
                         index=pivot.columns, columns=pivot.columns)
matrix


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [283]:
top5similar = matrix.T.sort_values(200,ascending=False)[200].iloc[1:6].reset_index()
top5similar

,CustomerID,200
0,3317,0.084959
1,1072,0.083972
2,3535,0.082403
3,1920,0.082100
4,3909,0.080920


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [284]:
top5 = group.loc[list(top5similar["CustomerID"])]
top5

SalesID  ProductID  Quantity
CustomerID ProductName                                                 
3317       Bay Leaf                        2240383        202         1
           Beef - Ground Medium            6007552         45         1
           Beef - Rib Eye Aaa              2367688        244         1
           Butter - Unsalted               4278889        402         1
           Cheese - Victor Et Berthold      703028        316         1
...                                            ...        ...       ...
3909       Wine - Charddonnay Errazuriz    9045484        450         2
           Wine - Fume Blanc Fetzer        3349679         76         1
           Wine - Magnotta, Merlot Sr Vqa  3566710        192         1
           Wine - Toasted Head             4134000        134         1
           Wine - Valpolicella Masi        3584400        296         1

[262 rows x 3 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [285]:
top5f = top5.groupby("ProductName").sum().sort_values("Quantity",ascending=False)["Quantity"]
top5f

ProductName
Soup - Campbells Bean Medley       4
Muffin - Carrot Individual Wrap    3
Bay Leaf                           3
Pork - Kidney                      3
Pepper - Black, Whole              3
                                  ..
Kiwi                               1
Knife Plastic - White              1
Lambcasing                         1
Lentils - Red, Dry                 1
Yoghurt Tubes                      1
Name: Quantity, Length: 207, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [286]:
recom5 = pivot.merge(top5f,on="ProductName").T.loc[[200,"Quantity"]].T.sort_values("Quantity",ascending=False)
recom5 = list(recom5[recom5[200]==0.0].head(5).reset_index()["ProductName"])
recom5

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [287]:
recoms = dict()
customers = list(set(data["CustomerID"]))
for customer in customers:
    step4 = matrix.T.sort_values(customer,ascending=False)[customer].iloc[1:6].reset_index()
    step5 = group.loc[list(step4["CustomerID"])]
    step6 = step5.groupby("ProductName").sum().sort_values("Quantity",ascending=False)["Quantity"]
    step7 = pivot.merge(step6,on="ProductName").T.loc[[200,"Quantity"]].T.sort_values("Quantity",ascending=False)
    step7b = list(step7[step7[200]==0.0].head(5).reset_index()["ProductName"])
    recoms.update({customer:step7b})


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [293]:
euclideo = pd.DataFrame(recoms)
euclideo.head(3)

,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Baking Powder,Beer - Rickards Red,Bread - Italian Roll With Herbs,Cinnamon Buns Sticky,Rabbit - Whole,Kellogs All Bran Bars,"Water - Mineral, Natural","Pepsi - Diet, 355 Ml",Sea Bass - Whole,Barramundi,...,Sea Bass - Whole,Ice Cream Bar - Oreo Cone,Sea Bass - Whole,Pecan Raisin - Tarts,Water - Aquafina Vitamin,Cheese - Victor Et Berthold,Grouper - Fresh,Pork - Kidney,Puree - Passion Fruit,Appetizer - Mushroom Tart
1,Guinea Fowl,"Shrimp - Baby, Warm Water","Bar Mix - Pina Colada, 355 Ml","Pasta - Penne, Rigate, Dry",Juice - Apple Cider,Appetizer - Sausage Rolls,Salmon Steak - Cohoe 8 Oz,Rice - Long Grain,Cocoa Butter,Cookies - Assorted,...,Pastry - Choclate Baked,"Wine - White, Mosel Gold",Spinach - Baby,Cod - Black Whole Fillet,Pants Custom Dry Clean,Oil - Safflower,"Chestnuts - Whole,canned","Pasta - Detalini, White, Fresh",Cake - Mini Cheesecake,Fenngreek Seed
2,Cocoa Butter,Assorted Desserts,Cod - Black Whole Fillet,Cheese Cloth No 100,Soupfoamcont12oz 112con,Cheese - Mozzarella,Spinach - Baby,Mussels - Cultivated,Pears - Bosc,Beef - Rib Eye Aaa,...,Lettuce - California Mix,Sobe - Tropical Energy,Ice Cream Bar - Oreo Cone,Thermometer Digital,Sword Pick Asst,Cake - Cake Sheet Macaroon,Napkin White - Starched,"Bar Mix - Pina Colada, 355 Ml",Ecolab - Solid Fusion,Cheese - Taleggio D.o.p.


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [277]:
matrix_coseno = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'cosine'))), 
                         index=pivot.columns, columns=pivot.columns)


In [278]:
recoms_coseno = dict()
customers = list(set(data["CustomerID"]))
for customer in customers:
    step4 = matrix_coseno.T.sort_values(customer,ascending=False)[customer].iloc[1:6].reset_index()
    step5 = group.loc[list(step4["CustomerID"])]
    step6 = step5.groupby("ProductName").sum().sort_values("Quantity",ascending=False)["Quantity"]
    step7 = pivot.merge(step6,on="ProductName").T.loc[[200,"Quantity"]].T.sort_values("Quantity",ascending=False)
    step7b = list(step7[step7[200]==0.0].head(5).reset_index()["ProductName"])
    recoms_coseno.update({customer:step7b})


In [296]:
coseno = pd.DataFrame(recoms)
coseno

,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Baking Powder,Beer - Rickards Red,Bread - Italian Roll With Herbs,Cinnamon Buns Sticky,Rabbit - Whole,Kellogs All Bran Bars,"Water - Mineral, Natural","Pepsi - Diet, 355 Ml",Sea Bass - Whole,Barramundi,...,Sea Bass - Whole,Ice Cream Bar - Oreo Cone,Sea Bass - Whole,Pecan Raisin - Tarts,Water - Aquafina Vitamin,Cheese - Victor Et Berthold,Grouper - Fresh,Pork - Kidney,Puree - Passion Fruit,Appetizer - Mushroom Tart
1,Guinea Fowl,"Shrimp - Baby, Warm Water","Bar Mix - Pina Colada, 355 Ml","Pasta - Penne, Rigate, Dry",Juice - Apple Cider,Appetizer - Sausage Rolls,Salmon Steak - Cohoe 8 Oz,Rice - Long Grain,Cocoa Butter,Cookies - Assorted,...,Pastry - Choclate Baked,"Wine - White, Mosel Gold",Spinach - Baby,Cod - Black Whole Fillet,Pants Custom Dry Clean,Oil - Safflower,"Chestnuts - Whole,canned","Pasta - Detalini, White, Fresh",Cake - Mini Cheesecake,Fenngreek Seed
2,Cocoa Butter,Assorted Desserts,Cod - Black Whole Fillet,Cheese Cloth No 100,Soupfoamcont12oz 112con,Cheese - Mozzarella,Spinach - Baby,Mussels - Cultivated,Pears - Bosc,Beef - Rib Eye Aaa,...,Lettuce - California Mix,Sobe - Tropical Energy,Ice Cream Bar - Oreo Cone,Thermometer Digital,Sword Pick Asst,Cake - Cake Sheet Macaroon,Napkin White - Starched,"Bar Mix - Pina Colada, 355 Ml",Ecolab - Solid Fusion,Cheese - Taleggio D.o.p.
3,Thermometer Digital,Knife Plastic - White,Cup - Translucent 7 Oz Clear,"Oil - Shortening,liqud, Fry",Anchovy Paste - 56 G Tube,Cheese Cloth No 100,"Cheese - Brie, Triple Creme",Cookie - Dough Variety,Sausage - Liver,"Pasta - Penne, Rigate, Dry",...,Wine - Redchard Merritt,Soupcontfoam16oz 116con,Onion Powder,Beef Wellington,Cake - Box Window 10x10x2.5,"Lamb - Pieces, Diced",Cheese - Cambozola,"Cheese - Boursin, Garlic / Herbs",Wine - Redchard Merritt,"Chestnuts - Whole,canned"
4,Muffin Chocolate Individual Wrap,Towels - Paper / Kraft,"Veal - Brisket, Provimi,bnls",Cookies - Assorted,Sprouts - Baby Pea Tendrils,"Nut - Chestnuts, Whole",Wine - Vineland Estate Semi - Dry,Vaccum Bag 10x13,Clam Nectar,Table Cloth 54x72 White,...,Isomalt,Vol Au Vents,Beets - Mini Golden,Oil - Shortening - All - Purpose,Arizona - Green Tea,Sea Bass - Whole,Sprouts - Alfalfa,Beans - Kidney White,Mussels - Frozen,"Bread - Roll, Soft White Round"


In [288]:
euclideo

,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Wine - White Cab Sauv.on,Ezy Change Mophandle,Pasta - Orecchiette,Wine - White Cab Sauv.on,Whmis - Spray Bottle Trigger,Mustard - Seed,Mustard - Seed,Wine - Chablis 2003 Champs,Phyllo Dough,Mustard - Seed,...,Sardines,"Wine - White, Mosel Gold",Tea - Decaf Lipton,"Sole - Dover, Whole, Fresh",Water - Aquafina Vitamin,Wine - Crozes Hermitage E.,Yogurt - French Vanilla,"Wine - Red, Colio Cabernet","Mushroom - Trumpet, Dry",Grapes - Red
1,Browning Caramel Glace,Anchovy Paste - 56 G Tube,Wanton Wrap,Wine - Chablis 2003 Champs,"Nut - Chestnuts, Whole",Pail With Metal Handle 16l White,Pail With Metal Handle 16l White,Peas - Frozen,Pears - Bosc,Wine - Redchard Merritt,...,"Beans - Kidney, Canned",Cheese - Wine,Pail With Metal Handle 16l White,Cake - Mini Cheesecake,Veal - Osso Bucco,Hersey Shakes,Island Oasis - Mango Daiquiri,Coffee - Dark Roast,"Oil - Shortening,liqud, Fry",Clam Nectar
2,Tea - Herbal Sweet Dreams,Pastry - Choclate Baked,Flavouring - Orange,Guinea Fowl,Wasabi Powder,Spinach - Baby,Bandage - Flexible Neon,"Placemat - Scallop, White",Pork - Belly Fresh,Pail With Metal Handle 16l White,...,Browning Caramel Glace,Tomato - Tricolor Cherry,Mussels - Frozen,Ecolab - Mikroklene 4/4 L,Garbag Bags - Black,"Soup - Campbells, Cream Of",Tia Maria,Bandage - Flexible Neon,"Cheese - Brie,danish","Crab - Dungeness, Whole"
3,Pastry - Choclate Baked,Milk - 2%,Tuna - Salad Premix,Cinnamon Buns Sticky,Salmon Steak - Cohoe 8 Oz,Juice - V8 Splash,Initation Crab Meat,Bandage - Flexible Neon,Fondant - Icing,Thermometer Digital,...,Sun - Dried Tomatoes,Pasta - Angel Hair,Thermometer Digital,Water - Aquafina Vitamin,Mangoes,Wine - Sogrape Mateus Rose,Cheese - Wine,Muffin Mix - Blueberry,Beer - Rickards Red,Pastry - Butterscotch Baked
4,Rice - Jasmine Sented,Sword Pick Asst,Garbage Bags - Clear,Peas - Frozen,Mussels - Cultivated,"Wine - White, Schroder And Schyl",Hersey Shakes,Cookie - Dough Variety,Jolt Cola - Electric Blue,Bread Crumbs - Panko,...,Juice - Apple Cider,Scampi Tail,Cheese - Parmesan Cubes,"Cheese - Brie, Triple Creme",Milk Powder,Cheese - Cottage Cheese,Garlic - Elephant,Beer - Blue,Sword Pick Asst,Vol Au Vents
